In [ ]:
import uiautomator2 as u2
import logging
import pymongo_management as pymongo_management
import toolfacebook_lib as toolfacebook_lib
from util import log_message 
import xml.etree.ElementTree as ET
import time
from lxml import etree

def connect_and_open_facebook(device_id):
    try:
        # Kết nối thiết bị qua USB
        driver = u2.connect(device_id)
        log_message(f"✅ Kết nối thành công với thiết bị: {device_id}")

        # Mở ứng dụng Facebook
        driver.app_start("com.facebook.katana", ".LoginActivity")
        log_message(f"🚀 Đã mở ứng dụng Facebook trên thiết bị: {device_id}")

        return driver
    except Exception as e:
        log_message(f"❌ Lỗi khi kết nối hoặc mở Facebook: {e}", level="ERROR")
        return None

In [ ]:
driver = connect_and_open_facebook("1ac1d26f0507")

In [ ]:
def comment_recruitment_post(driver, user_id):
    comment = pymongo_management.get_comment(user_id)
    log_message(comment[0]['message'], comment[1])
    if 'comment' in comment[0]:
        comment = comment[0]
        link = comment.get("link")
        comment = comment.get("comment", None)
        toolfacebook_lib.redirect_to(driver, link)
        driver(text="Bình luận").click()
        driver.send_keys(comment, clear=True)
        driver(description="Gửi").click()
        log_message(f"Bình luận thương hiệu: Đã bình luận: {comment}", logging.INFO)
        toolfacebook_lib.back_to_facebook(driver)
comment_recruitment_post(driver, "test")

In [ ]:
def join_group(driver, user_id, group_link):
    toolfacebook_lib.redirect_to(driver, "https://facebook.com/" + group_link)
    time.sleep(2)
    joined_group = driver(textContains="đã tham gia nhóm")
    if joined_group.exists:
        result = pymongo_management.update_joined_accounts(user_id, group_link)
        log_message(result[0]['message'], result[1])
        toolfacebook_lib.back_to_facebook(driver)
        return

    join_button_clicked = False
    join_button = driver(textContains="Tham gia nhóm")
    if join_button.exists:
        join_button.click()
        join_button_clicked = True

        joined_group = driver(textContains="đã tham gia nhóm")
        if joined_group.exists:
            result = pymongo_management.update_joined_accounts(user_id, group_link)
            log_message(result[0]['message'], result[1])
            toolfacebook_lib.back_to_facebook(driver)
            return

    if not toolfacebook_lib.click_template(driver, "answer_question") and not join_button_clicked:
        toolfacebook_lib.back_to_facebook(driver)
        return
    time.sleep(2)
    xml_dump = driver.dump_hierarchy()
    root = ET.fromstring(xml_dump)

    visible_texts = []
    nodes = []

    for node in root.iter():
        visible = node.attrib.get("visible-to-user", "false") == "true"
        text = node.attrib.get("text", "").strip()

        if visible and text:
            visible_texts.append(text)
            nodes.append(node)

    all_questions_answered = True
    if "Trả lời câu hỏi" in visible_texts:
        indexes = [i for i, text in enumerate(visible_texts) if (text == "Viết câu trả lời..." or text == "Bạn có thể chọn nhiều đáp án" or text == "Bạn có thể chọn 1 đáp án" or text == "Gửi") and visible_texts[i-1] != "Viết câu trả lời..."]
        indexes[-1] += 1
        for i in range(len(indexes) - 1):
            answers = []
            for j in range(indexes[i] + 1, indexes[i + 1] - 1):
                answers.append(visible_texts[j])
            pymongo_management.upload_question(group_link, visible_texts[indexes[i] - 1], visible_texts[indexes[i]], answers)
            answer, log = pymongo_management.get_answer(group_link, visible_texts[indexes[i] - 1])
            log_message(answer['message'], log)
            if "answer" in answer:
                if visible_texts[indexes[i]] == "Viết câu trả lời...":
                    bounds = nodes[indexes[i]].attrib.get("bounds")
                    bounds = toolfacebook_lib.parse_number(bounds)
                    driver.click((bounds[0] + bounds[2]) // 2, (bounds[1] + bounds[3]) // 2)
                    driver.send_keys(answer['answer'], clear=True)
                    driver.press("back")  # Nhấn nút back để đóng bàn phím
                elif visible_texts[indexes[i]] == "Bạn có thể chọn nhiều đáp án":
                    for i in range(indexes[i] + 1, indexes[i + 1] - 1):
                        if visible_texts[i] in answer['answer']:
                            bounds = nodes[i].attrib.get("bounds")
                            bounds = toolfacebook_lib.parse_number(bounds)
                            driver.click((bounds[0] + bounds[2]) // 2, (bounds[1] + bounds[3]) // 2)
                elif visible_texts[indexes[i]] == "Bạn có thể chọn 1 đáp án":
                    for i in range(indexes[i] + 1, indexes[i + 1] - 1):
                        if visible_texts[i] == answer['answer']:
                            bounds = nodes[i].attrib.get("bounds")
                            bounds = toolfacebook_lib.parse_number(bounds)
                            driver.click((bounds[0] + bounds[2]) // 2, (bounds[1] + bounds[3]) // 2)
                            break
            else:
                all_questions_answered = False
    if all_questions_answered:
        driver(description="Gửi").click()
        log_message(pymongo_management.update_temp_joined_accounts(user_id, group_link)['message'])
    else:
        driver(resourceId="com.android.systemui:id/back").click()
        driver(resourceId="com.facebook.katana:id/(name removed)", text="THOÁT").click()
    toolfacebook_lib.back_to_facebook(driver)
# join_group(driver, "test", "groups/24635611929409028")

In [ ]:
def check_unapproved_groups(driver, user_id):
    unapproved_groups = pymongo_management.get_unapproved_groups(user_id)
    for group in unapproved_groups:
        join_group(driver, user_id, group['Link'])

check_unapproved_groups(driver, "test")

In [ ]:
def post_to_group(driver, command_id, group_link, content, files=None):
    if files:
        for file in files:
            toolfacebook_lib.push_file_to_device(driver.serial, file)
        driver.app_start("com.miui.gallery")
    toolfacebook_lib.redirect_to(driver, "https://facebook.com/" + group_link)
    time.sleep(2)
    joined_group = driver(textContains="đã tham gia nhóm")
    if not joined_group.exists:
        log_message(f"Đăng bài lên nhóm: Chưa tham gia nhóm {group_link}", logging.WARNING)
        toolfacebook_lib.back_to_facebook(driver)
        return

    post_button = driver(text="Bạn viết gì đi...")
    if post_button.exists:
        post_button.click()
        time.sleep(2)
        width, height = driver.window_size()
        center_x = width // 2
        center_y = height // 2
        driver.click(center_x, center_y)
        driver.send_keys(content, clear=True)
        if files:
            driver(description="Ảnh/video").click()
            time.sleep(2)
            driver(description="Chọn nhiều file").click()
            for i in range(len(files)):
                driver.xpath(f'//android.widget.GridView/android.widget.Button[{i + 1}]/android.widget.Button[1]').click()
            driver(text="Tiếp").click()
            time.sleep(2)
            try:
                driver(scrollable=True).scroll.vert.backward()
            except:
                pass
        driver(text="ĐĂNG").click()
        log_message(f"Đăng bài lên nhóm: Đã đăng bài viết vào nhóm {group_link}", logging.INFO)
    else:
        log_message("Đăng bài lên nhóm: Không tìm thấy nút Tạo bài viết", logging.WARNING)
    toolfacebook_lib.back_to_facebook(driver)
    if files:
        for file in files:
            toolfacebook_lib.delete_file(driver.serial, file)
    pymongo_management.execute_command(command_id)

commands = pymongo_management.get_commands("123")
for command in commands:
    if command['type'] == 'post_to_group':
        params = command.get("params", {})
        post_to_group(driver, command['_id'], params.get("group_link", ""), params.get("content", ""), params.get("files", []))
    if command['type'] == 'join_group':
        params = command.get("params", {})
        join_group(driver, command['user_id'], params.get("group_link", ""))

In [ ]:
def check_post(driver, user_id):
    posts = pymongo_management.get_unapproved_posts(user_id)
    for post in posts:
        toolfacebook_lib.redirect_to(driver, "https://facebook.com/" + post['group_link'])
        time.sleep(1)
        driver(description="Bạn").click()
        while toolfacebook_lib.is_screen_changed(driver):
            xml = driver.dump_hierarchy()
            tree = etree.fromstring(xml.encode("utf-8"))

            # Tìm nút "Thích"
            like_nodes = tree.xpath("//node[@text='Nút Thích. Hãy nhấn đúp và giữ để bày tỏ cảm xúc về bình luận.' or @text='Đã nhấn nút Thích. Nhấn đúp và giữ để thay đổi cảm xúc.']")
            grandparents = []

            for like in like_nodes:
                parent = like.getparent()
                grandparent = parent.getparent() if parent is not None else None

                if grandparent is not None:
                    grandparents.append(grandparent)
            found = False
            for grandparent in grandparents:
                status = "Đang chờ duyệt"
                for child in grandparent.iter():
                    text = child.attrib.get("text", "")
                    if text == post['content']:
                        found = True
                    if text == "Bình luận":
                        status = "Đã đăng thành công"
                if found:
                    break
            if found:
                if post.get('link', "") == "":
                    post_link = toolfacebook_lib.extract_post_link(driver, grandparent)
                else:
                    post_link = post.get('link', "")
                log_message(pymongo_management.update_post_status(post['_id'], status, post_link)[0]['message'], logging.INFO)
                break
            driver.swipe_ext("up", scale=0.8)
    time.sleep(1)
    toolfacebook_lib.back_to_facebook(driver)
check_post(driver, "123")

In [ ]:
def crawl_post_comment(driver, post_link):
    toolfacebook_lib.redirect_to(driver, post_link)
    location_by_type = []
    index = 1
    raw_comments = {}
    temp_parents = {}
    links = {}
    toolfacebook_lib.change_comment_display_mode(driver)
    driver.swipe_ext("down", scale=0.8)
    time.sleep(1)
    toolfacebook_lib.expand_collapse_section(driver)
    while True:
        node = f'//androidx.recyclerview.widget.RecyclerView/android.view.ViewGroup[{index}]/android.view.ViewGroup[1]/android.view.ViewGroup[1]'
        content_node = driver.xpath(f'//androidx.recyclerview.widget.RecyclerView/android.view.ViewGroup[{index}]/android.view.ViewGroup[1]/android.view.ViewGroup[2]')
        if not driver.xpath(node).exists or not content_node.exists:
            driver.swipe_ext("up", scale=0.8)
            index = 1
            time.sleep(1)
            toolfacebook_lib.expand_collapse_section(driver)
            time.sleep(1)
            if not toolfacebook_lib.is_screen_changed(driver, threshold=0.9):
                break
            continue
        
        result = toolfacebook_lib.extract_comment_info(driver, node, raw_comments, links)
        if result:
            time.sleep(1)
            bounds = content_node.get().attrib.get("bounds")
            bounds = toolfacebook_lib.parse_number(bounds)
            if bounds[0] not in location_by_type and bounds[0] > 0:
                location_by_type.append(bounds[0])
                location_by_type.sort()
            level = len([x for x in location_by_type if x < bounds[0]])
            temp_parents[level] = (result['name'], result['comment'])
            if "time" in result:
                save_result = pymongo_management.save_post_comment(post_link, result['name'], result['comment'], result['time'], level, temp_parents[level - 1][0] if level > 0 else None, temp_parents[level - 1][1] if level > 0 else None)
                log_message(f"{driver.serial}: {save_result[0]['message']}", save_result[1])
                raw_comments[result['comment']] = result['name']
        index += 1
crawl_post_comment(driver, "https://www.facebook.com/share/p/19iY4FCKcL/")

In [ ]:
toolfacebook_lib.redirect_to(driver, "https://www.facebook.com/share/p/19iY4FCKcL/")